In [10]:
!pip install yfinance

In [2]:
pip install plotly

In [3]:
pip install dash

**FINAL CODE WITH ALL FEATURES**
### <a></a>FINAL MODEL


In [13]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import LSTM, Dense
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State
import dash_table
from sklearn.metrics import mean_absolute_error, mean_squared_error
import webbrowser

# Initialize Dash app
app = Dash(__name__)
# Run the app on a new thread to open it in the browser automatically
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")  # This will open the app in a new browser window

company_tickers= [
        "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK.B", "UNH", "V", "HD", "DIS", "PYPL", "NFLX", "PEP", "XOM", 
        "PFE", "MA", "JNJ", "NVDA", "VZ", "BABA", "CVX", "INTC", "KO", "WMT", "BA", "CSCO", "ABT", "CMCSA", "ADBE", "T", "INTU", "AMD",
        "MCD", "CAT", "AMGN", "GS", "IBM", "C", "JPM", "LMT", "RTX", "NKE", "GE", "SBUX", "BA", "USB", "GS", "MS", "MMM", "AIG", "AXP",
        "NSC", "CB", "DUK", "MS", "ZTS", "MO", "BIIB", "CHTR", "BMY", "LOW", "PYPL", "AMT", "TMO", "FISV", "COP", "CI", "SPGI", "PXD", 
        "APD", "VRTX", "MNST", "PGR", "BAX", "GD", "VLO", "ECL", "CSX", "TGT", "F", "MAA", "DISCA", "SYY", "NOC", "ILMN", "BDX", "REGN", 
        "ADI", "MSCI", "OXY", "AON", "STZ", "DE", "CME", "CTSH", "AEP", "CLX", "HUM", "SIVB", "SCHW", "GILD", "LRCX", "WBA", "UNP", "WDC",
        "HON", "ETN", "DLR", "TMUS", "AMGN", "LMT", "T", "MKTX", "ZBRA", "KMB", "BA", "PG", "XEL", "IBM", "DHR", "AMAT", "MSFT", "COST",
        "SBUX", "HCA", "PEP", "CVS", "CL", "FIS", "MCO", "TRV", "CB", "RTN", "HON", "EXPE", "NTAP", "KSU", "CAT", "MDT", "HCA", "HCA",
        "EFX", "V", "NOW", "NLOK", "KEYS", "TMUS", "SE", "TXN", "BWA", "VRSN", "NVIDIA", "UAL", "AMZN", "BA", "BHP", "TRIP", "BAX", "MKC",
        "ULTA", "MOS", "CTXS", "PEP", "HON", "GSK", "JPM", "DIS", "MMM", "LULU", "CSCO", "PG", "EXPE", "UAL", "MRK", "AIG", "EQIX", 
        "UPS", "ALGN", "INTC", "AMAT", "NFLX", "NVDA", "CSX", "AIG", "PGR", "COF", "SBUX", "LHX", "AMGN", "MRNA", "NTAP", "TRV", "MDT",
        "VLO", "WMT", "ZTS", "BA", "DLTR", "PFE", "BIDU", "HPE", "EBAY", "AMZN", "F", "IDXX", "LUV", "VLO", "AON", "LUV", "NSC", "EXPE",
        "CTSH", "MCK", "GPN", "GS", "KMB", "MNST", "NKE", "TGT", "MO", "CI", "SPGI", "ABBV", "FTNT", "MTD", "BAX", "V", "LMT", "ZBH", 
        "ATVI", "ADM", "DE", "GS", "ORLY", "ZTS", "TSLA", "DHR", "SPGI", "SWKS", "LVS", "ADBE", "NFLX", "ALXN", "FISV", "AIG", "ALL",
        "NVDA", "BA", "AAPL", "INTC", "MELI", "KLAC", "TSLA", "CVX", "NOC", "PEP", "WMT", "HPQ", "CSX", "VLO", "SIVB", "SBUX", "MMM",
        "PFE", "MRK", "IBM", "CAT", "SYY", "GOOG", "WDC", "NVDA", "AMZN", "INTU", "PLD", "EXPE", "MS", "PM", "NKE", "AMT", "MNST", "MSCI",
        "SNPS", "DHR", "AXP", "HCA", "AMGN", "BAX", "V", "PFE", "HPQ", "SYY", "MDT", "COP", "F", "AMD", "FIS", "MA", "AON", "GIS", "MRK",
        "MCD", "GILD", "JNJ", "CSCO", "PG", "LMT", "ADBE", "CHTR", "CL", "DUK", "PYPL", "IBM", "MDT", "NSC", "LRCX", "WBA", "ZM", "TMO",
        "AEP", "MSCI", "SIVB", "STZ", "CSX", "GOOGL", "AIG", "VLO", "CVX", "KMB", "EXPE", "EL", "INTC", "C", "SNPS", "TRV", "XOM", "GM",
        "WMT", "HD", "SQ", "XEL", "NKE", "BLK", "GS", "ADBE", "FISV", "UNP", "TMO", "DIS", "V", "USB", "ETN", "BA", "PEP", "CHD", "MCO",
        "TGT", "HUM", "GM", "IP", "BIDU", "DHR", "COST", "LULU", "MAA", "TMUS", "EBAY", "AMGN", "F", "MCK", "UNP", "MKTX", "RSG", "EXPE"
    ]
# Fetch stock data
def get_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    if stock_data.empty:
        raise ValueError("No data fetched for the given ticker and date range.")
    return stock_data

# ARIMA prediction
def arima_predict(data, future_days=7):
    model = ARIMA(data['Close'], order=(7,0,6))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=future_days)
    return forecast

# Prepare data for LSTM and XGBoost
def prepare_data(data, time_step=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    X, y = [], []
    for i in range(time_step, len(data_scaled)):
        X.append(data_scaled[i-time_step:i, 0])
        y.append(data_scaled[i, 0])
    return np.array(X), np.array(y), scaler

# LSTM prediction
def lstm_predict(data, time_step=60, future_days=5):
    X, y, scaler = prepare_data(data, time_step)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, y, epochs=5, batch_size=32, verbose=0)

    future_predictions = []
    last_sequence = X[-1]

    for _ in range(future_days):
        pred = model.predict(last_sequence.reshape(1, time_step, 1))
        future_predictions.append(pred[0, 0])
        last_sequence = np.append(last_sequence[1:], pred[0, 0])

    return scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1)).flatten()

# XGBoost prediction
def xgboost_predict(data, time_step=60, future_days=5):
    X, y, scaler = prepare_data(data, time_step)
    model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.01, n_estimators=100, max_depth=5, subsample=0.8)
    model.fit(X, y)

    future_predictions = []
    last_sequence = X[-1]

    for _ in range(future_days):
        pred = model.predict(last_sequence.reshape(1, time_step))
        future_predictions.append(pred[0])
        last_sequence = np.append(last_sequence[1:], pred[0])

    return scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1)).flatten()
# Function to compute MAE and RMSE
def compute_metrics(true_values, predicted_values):
    mae = mean_absolute_error(true_values, predicted_values)
    rmse = np.sqrt(mean_squared_error(true_values, predicted_values))
    return mae, rmse

# Update the callback or testing workflow
def evaluate_models(data, time_step=60):
    # Split data into training and testing
    train_size = int(len(data) * 0.8)
    train_data, test_data = data[:train_size], data[train_size:]
    
    # Train ARIMA
    arima_forecast = arima_predict(train_data, future_days=len(test_data))
    arima_mae, arima_rmse = compute_metrics(test_data['Close'], arima_forecast)
    
    # Train and test LSTM
    lstm_forecast = lstm_predict(train_data, time_step=time_step, future_days=len(test_data))
    lstm_mae, lstm_rmse = compute_metrics(test_data['Close'], lstm_forecast)
    
    # Train and test XGBoost
    xgboost_forecast = xgboost_predict(train_data, time_step=time_step, future_days=len(test_data))
    xgboost_mae, xgboost_rmse = compute_metrics(test_data['Close'], xgboost_forecast)
    
    # Combine metrics for comparison
    metrics = {
        "ARIMA": {"MAE": arima_mae, "RMSE": arima_rmse},
        "LSTM": {"MAE": lstm_mae, "RMSE": lstm_rmse},
        "XGBoost": {"MAE": xgboost_mae, "RMSE": xgboost_rmse}
    }
    return metrics
# Generate recommendations
def generate_recommendations(current_price, predictions):
    recommendations = {}
    for model_name, preds in predictions.items():
        predicted_price = preds.iloc[-1] if isinstance(preds, pd.Series) else preds[-1]
        percent_change = ((predicted_price - current_price) / current_price) * 100
        recommendation = "Buy" if percent_change > 2 else "Hold/Wait"
        recommendations[model_name] = {"Recommendation": recommendation, "Percent Change": percent_change}
    return recommendations

app.layout = html.Div([
    html.H1("Stock Prediction with ARIMA, LSTM, and XGBoost"),
    html.Div([
        html.Label("Select Stock Ticker:"),
        dcc.Dropdown(
            id="ticker",
            options=[{"label": ticker, "value": ticker} for ticker in company_tickers],
            value="AAPL",  # Default value
            style={'marginRight': '10px'}
        ),
    ], style={'marginBottom': '10px'}),


    html.Div([
        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id="date-picker-range",
            start_date="2023-01-01",
            end_date="2024-11-11",
            display_format='YYYY-MM-DD',
            style={'marginBottom': '20px'}
        ),
    ]),

    html.Div([
        html.Label("Select Prediction Range:"),
        dcc.Dropdown(
            id="range-selector",
            options=[
                {"label": "1 Week", "value": "week"},
                {"label": "1 Month", "value": "month"},
                {"label": "3 Months", "value": "3months"}
            ],
            value="week",
            style={'width': '200px', 'marginBottom': '20px'}
        ),
    ]),

    html.Button('Submit', id='submit-button', n_clicks=0, style={'marginBottom': '20px'}),

    dcc.Graph(id="stock-graph"),

    html.Div([
        html.H2("Model Evaluation Metrics"),
        dash_table.DataTable(
            id='metrics-table',
            columns=[
                {"name": "Model", "id": "Model"},
                {"name": "MAE", "id": "MAE"},
                {"name": "RMSE", "id": "RMSE"}
            ],
            data=[],  # To be populated by callback
            style_table={'width': '60%', 'marginTop': '20px'},
            style_cell={'textAlign': 'center', 'padding': '5px'},
            style_header={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'white', 'fontWeight': 'bold'},
            style_data={'backgroundColor': 'rgb(50, 50, 50)', 'color': 'white'}
        )
    ]),

    html.H2("Recommendations"),
    dash_table.DataTable(
        id='recommendations-table',
        columns=[{"name": "Model", "id": "Model"}, {"name": "Recommendation", "id": "Recommendation"}, {"name": "Percent Change", "id": "Percent Change"}],
        data=[], style_table={'width': '50%', 'marginTop': '20px'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
        style_header={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'white', 'fontWeight': 'bold'},
        style_data={'backgroundColor': 'rgb(50, 50, 50)', 'color': 'white'}
    )
])


@app.callback(
    [Output("stock-graph", "figure"), 
     Output("recommendations-table", "data"),
     Output("metrics-table", "data")],
    [Input("submit-button", "n_clicks")],
    [State("ticker", "value"), State("date-picker-range", "start_date"), State("date-picker-range", "end_date"), State("range-selector", "value")]
)
def update_graph(n_clicks, ticker, start_date, end_date, selected_range):
    if n_clicks == 0:
        return go.Figure(), [], []

    data = get_stock_data(ticker, start_date, end_date)
    current_price = data['Close'].iloc[-1]

    future_days = {"week": 7, "month": 30, "3months": 90}[selected_range]
    predictions = {
        "ARIMA": arima_predict(data, future_days=future_days),
        "LSTM": lstm_predict(data, future_days=future_days),
        "XGBoost": xgboost_predict(data, future_days=future_days)
    }

    recommendations = generate_recommendations(current_price, predictions)
    table_data = [{"Model": model, "Recommendation": rec["Recommendation"], "Percent Change": f"{rec['Percent Change']:.2f}%"} for model, rec in recommendations.items()]

    metrics = evaluate_models(data)  # Compute metrics for each model
    metrics_data = [{"Model": model, "MAE": f"{values['MAE']:.2f}", "RMSE": f"{values['RMSE']:.2f}"} for model, values in metrics.items()]

    # Create the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Historical Data'))
    for model, preds in predictions.items():
        fig.add_trace(go.Scatter(x=[data.index[-1] + timedelta(days=i) for i in range(1, future_days+1)], y=preds, mode='lines+markers', name=f"{model} Prediction"))

    fig.update_layout(title=f"Stock Price Prediction for {ticker.upper()}", xaxis_title="Date", yaxis_title="Price (USD)", legend_title="Model")

    return fig, table_data, metrics_data


if __name__ == "__main__":
    open_browser()  # Open browser immediately
    app.run_server(debug=False)

**FINAL CODE FOR STOCK DATA**
### <a></a>STOCK DATA


In [14]:
import dash
from dash import dcc, html, Input, Output, State, dash_table
import yfinance as yf
import pandas as pd

# List of 500 company tickers
company_tickers = [
   "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK.B", "UNH", "V", "HD", "DIS", "PYPL", "NFLX", "PEP", "XOM", 
        "PFE", "MA", "JNJ", "NVDA", "VZ", "BABA", "CVX", "INTC", "KO", "WMT", "BA", "CSCO", "ABT", "CMCSA", "ADBE", "T", "INTU", "AMD",
        "MCD", "CAT", "AMGN", "GS", "IBM", "C", "JPM", "LMT", "RTX", "NKE", "GE", "SBUX", "BA", "USB", "GS", "MS", "MMM", "AIG", "AXP",
        "NSC", "CB", "DUK", "MS", "ZTS", "MO", "BIIB", "CHTR", "BMY", "LOW", "PYPL", "AMT", "TMO", "FISV", "COP", "CI", "SPGI", "PXD", 
        "APD", "VRTX", "MNST", "PGR", "BAX", "GD", "VLO", "ECL", "CSX", "TGT", "F", "MAA", "DISCA", "SYY", "NOC", "ILMN", "BDX", "REGN", 
        "ADI", "MSCI", "OXY", "AON", "STZ", "DE", "CME", "CTSH", "AEP", "CLX", "HUM", "SIVB", "SCHW", "GILD", "LRCX", "WBA", "UNP", "WDC",
        "HON", "ETN", "DLR", "TMUS", "AMGN", "LMT", "T", "MKTX", "ZBRA", "KMB", "BA", "PG", "XEL", "IBM", "DHR", "AMAT", "MSFT", "COST",
        "SBUX", "HCA", "PEP", "CVS", "CL", "FIS", "MCO", "TRV", "CB", "RTN", "HON", "EXPE", "NTAP", "KSU", "CAT", "MDT", "HCA", "HCA",
        "EFX", "V", "NOW", "NLOK", "KEYS", "TMUS", "SE", "TXN", "BWA", "VRSN", "NVIDIA", "UAL", "AMZN", "BA", "BHP", "TRIP", "BAX", "MKC",
        "ULTA", "MOS", "CTXS", "PEP", "HON", "GSK", "JPM", "DIS", "MMM", "LULU", "CSCO", "PG", "EXPE", "UAL", "MRK", "AIG", "EQIX", 
        "UPS", "ALGN", "INTC", "AMAT", "NFLX", "NVDA", "CSX", "AIG", "PGR", "COF", "SBUX", "LHX", "AMGN", "MRNA", "NTAP", "TRV", "MDT",
        "VLO", "WMT", "ZTS", "BA", "DLTR", "PFE", "BIDU", "HPE", "EBAY", "AMZN", "F", "IDXX", "LUV", "VLO", "AON", "LUV", "NSC", "EXPE",
        "CTSH", "MCK", "GPN", "GS", "KMB", "MNST", "NKE", "TGT", "MO", "CI", "SPGI", "ABBV", "FTNT", "MTD", "BAX", "V", "LMT", "ZBH", 
        "ATVI", "ADM", "DE", "GS", "ORLY", "ZTS", "TSLA", "DHR", "SPGI", "SWKS", "LVS", "ADBE", "NFLX", "ALXN", "FISV", "AIG", "ALL",
        "NVDA", "BA", "AAPL", "INTC", "MELI", "KLAC", "TSLA", "CVX", "NOC", "PEP", "WMT", "HPQ", "CSX", "VLO", "SIVB", "SBUX", "MMM",
        "PFE", "MRK", "IBM", "CAT", "SYY", "GOOG", "WDC", "NVDA", "AMZN", "INTU", "PLD", "EXPE", "MS", "PM", "NKE", "AMT", "MNST", "MSCI",
        "SNPS", "DHR", "AXP", "HCA", "AMGN", "BAX", "V", "PFE", "HPQ", "SYY", "MDT", "COP", "F", "AMD", "FIS", "MA", "AON", "GIS", "MRK",
        "MCD", "GILD", "JNJ", "CSCO", "PG", "LMT", "ADBE", "CHTR", "CL", "DUK", "PYPL", "IBM", "MDT", "NSC", "LRCX", "WBA", "ZM", "TMO",
        "AEP", "MSCI", "SIVB", "STZ", "CSX", "GOOGL", "AIG", "VLO", "CVX", "KMB", "EXPE", "EL", "INTC", "C", "SNPS", "TRV", "XOM", "GM",
        "WMT", "HD", "SQ", "XEL", "NKE", "BLK", "GS", "ADBE", "FISV", "UNP", "TMO", "DIS", "V", "USB", "ETN", "BA", "PEP", "CHD", "MCO",
        "TGT", "HUM", "GM", "IP", "BIDU", "DHR", "COST", "LULU", "MAA", "TMUS", "EBAY", "AMGN", "F", "MCK", "UNP", "MKTX", "RSG", "EXPE"
]
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

# Create dropdown options from ticker list
dropdown_options = [{"label": ticker, "value": ticker} for ticker in company_tickers]

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Stock Data Viewer"

# Layout
app.layout = html.Div([
    html.H1("Stock Data Viewer", style={'textAlign': 'center'}),

    # Inputs: Ticker Dropdown, Date Range, Submit Button
    html.Div([
        html.Label("Select Ticker Symbol:"),
        dcc.Dropdown(
            id="ticker-dropdown",
            options=dropdown_options,
            placeholder="Select a company...",
            style={'width': '100%'}
        ),
        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id="date-picker",
            start_date_placeholder_text="Start Date",
            end_date_placeholder_text="End Date",
            display_format="YYYY-MM-DD"
        ),
        html.Button("Submit", id="submit-button", n_clicks=0, style={'marginTop': '10px'}),
    ], style={'padding': '10px', 'width': '30%', 'margin': '0 auto'}),

    # Output: Stock Table
    html.Div([
        dash_table.DataTable(
            id="stock-table",
            columns=[
                {"name": "Date", "id": "Date"},
                {"name": "Open", "id": "Open"},
                {"name": "High", "id": "High"},
                {"name": "Low", "id": "Low"},
                {"name": "Close", "id": "Close"},
                {"name": "Adj Close", "id": "Adj Close"},
                {"name": "Volume", "id": "Volume"},
            ],
            style_table={'overflowX': 'auto', 'marginTop': '20px'},
            style_cell={'textAlign': 'center', 'padding': '5px'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
        )
    ], style={'padding': '10px', 'width': '80%', 'margin': '0 auto'})
])

# Callback to fetch and display stock data
@app.callback(
    Output("stock-table", "data"),
    [Input("submit-button", "n_clicks")],
    [State("ticker-dropdown", "value"),
     State("date-picker", "start_date"),
     State("date-picker", "end_date")]
)
def update_stock_table(n_clicks, ticker, start_date, end_date):
    if n_clicks == 0 or not ticker or not start_date or not end_date:
        return []

    # Fetch stock data using yfinance
    try:
        df = yf.download(ticker, start=start_date, end=end_date)
        if df.empty:
            return []

        # Reset index and format DataFrame for display
        df = df.reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')  # Format date
        return df.to_dict('records')  # Convert DataFrame to dictionary for Dash DataTable
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

# Run the app and open in a new browser window
if __name__ == "__main__":
    open_browser()  # Open browser immediately
    app.run_server(debug=False)

[*********************100%***********************]  1 of 1 completed
